In [ ]:
# Start writing code here...from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [ ]:
spotify_songs = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv')


Q1 = np.percentile(spotify_songs.duration_ms, 25,
                   interpolation = 'midpoint')
Q3 = np.percentile(spotify_songs.duration_ms, 75,
                   interpolation = 'midpoint')
IQR = Q3 - Q1

"""
mask = (spotify_songs.duration_ms < (Q3+1.5*IQR))
mask2 = spotify_songs.duration_ms > (Q1-1.5*IQR)
spotify_data_cleaned = spotify_songs.loc[mask]
spotify_data_cleaned = spotify_data_cleaned.loc[mask2]
"""
spotify_data_cleaned = spotify_songs
spotify_data_cleaned.track_album_release_date.str[:4]
spotify_data_cleaned.track_album_release_date.str[5:7]
spotify_data_cleaned = spotify_data_cleaned.assign(
    track_album_release_month = spotify_data_cleaned.track_album_release_date.str[5:7],
    track_album_release_year = spotify_data_cleaned.track_album_release_date.str[:4],
)
spotify_data_cleaned.drop(columns="track_album_release_date",inplace=True)
spotify_data_cleaned.track_album_release_year = spotify_data_cleaned.track_album_release_year.astype("int64")
"""
mask = (spotify_data_cleaned.track_popularity > 0)
spotify_data_cleaned = spotify_data_cleaned.loc[mask]
"""


'\nmask = (spotify_data_cleaned.track_popularity > 0)\nspotify_data_cleaned = spotify_data_cleaned.loc[mask]\n'

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def normalisation(data):
    columns = ["track_popularity","danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo", "duration_ms", "track_album_release_year"]
    data_norme = data[columns]
    scaler = MinMaxScaler(feature_range=(0,1))
    data_norme = pd.DataFrame(scaler.fit_transform(data_norme), columns=columns)
    return data_norme


In [ ]:
from scipy.stats import shapiro

def shapiro_test(genre):
    mask = spotify_data_cleaned.playlist_genre == genre
    data= spotify_data_cleaned.loc[mask]
    data = normalisation(data)
    return(shapiro(data))
shapiro_test("edm")

/shared-libs/python3.7/py/lib/python3.7/site-packages/scipy/stats/morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


ShapiroResult(statistic=0.9340165257453918, pvalue=0.0)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

def analyse_2_genre(genre1,genre2):
    mask_1 = spotify_data_cleaned.playlist_genre == genre1
    mask_2 = spotify_data_cleaned.playlist_genre == genre2

    data_1= spotify_data_cleaned.loc[mask_1]
    data_2 = spotify_data_cleaned.loc[mask_2]
    
    data= pd.concat([data_1,data_2])


    print(genre1 , " ", genre2)

    LDA(normalisation(data),data)
    QDA(normalisation(data),data)
    GNB(normalisation(data),data)

    print("-----------------------")

    return data,normalisation(data)

In [ ]:
def LDA(data_normed_quanti,data):
    xx = data_normed_quanti
    yy = data.playlist_genre
    x_train, x_test, y_train, y_test = train_test_split(xx, yy, train_size=0.66)
    lda = LinearDiscriminantAnalysis()
    lda.fit(x_train,y_train)
    print("LDA : " , lda.score(x_test,y_test))


In [ ]:
def QDA(data_normed_quanti,data):
    xx = data_normed_quanti
    yy = data.playlist_genre
    x_train, x_test, y_train, y_test = train_test_split(xx, yy, train_size=0.66)
    qda = QuadraticDiscriminantAnalysis()
    qda.fit(x_train,y_train)
    print("QDA : " , qda.score(x_test,y_test))


In [ ]:
def GNB(data_normed_quanti,data):
    xx = data_normed_quanti
    yy = data.playlist_genre
    x_train, x_test, y_train, y_test = train_test_split(xx, yy, train_size=0.66)
    gnb = GaussianNB()
    gnb.fit(x_train,y_train)
    print("GNB : " , gnb.score(x_test,y_test))


In [ ]:
confrontGenres = [["edm","rap"],["edm","rock"],["edm","r&b"],["edm","latin"],["edm","pop"],["rap","rock"],["rap","r&b"],["rap","latin"],["rap","pop"],["rock","r&b"],["rock","latin"],["rock","pop"],["r&b","latin"],["r&b","pop"],["latin","pop"]]

In [ ]:
for genre in confrontGenres:
    analyse_2_genre(genre[0],genre[1])

edm   rap
LDA :  0.8149164380144674
QDA :  0.8381142429533549
GNB :  0.822898478423547
-----------------------
edm   rock
LDA :  0.8871054039593366
QDA :  0.9023542001070091
GNB :  0.8715890850722311
-----------------------
edm   r&b
LDA :  0.8508457201435161
QDA :  0.8385443362378268
GNB :  0.844438749359303
-----------------------
edm   latin
LDA :  0.770483193277311
QDA :  0.7928046218487395
GNB :  0.7851890756302521
-----------------------
edm   pop
LDA :  0.7308377896613191
QDA :  0.7557932263814616
GNB :  0.7394957983193278
-----------------------
rap   rock
LDA :  0.9128402529557328
QDA :  0.9175144349738795
GNB :  0.8823205938960682
-----------------------
rap   r&b
LDA :  0.7240199947382268
QDA :  0.7124440936595633
GNB :  0.6927124440936596
-----------------------
rap   latin
LDA :  0.7267332074453736
QDA :  0.7315888858915565
GNB :  0.6727812247100081
-----------------------
rap   pop
LDA :  0.7886072641755945
QDA :  0.7906976744186046
GNB :  0.7499346746799059
-------------

## Sur toutes les données normées

In [ ]:
LDA(normalisation(spotify_data_cleaned),spotify_data_cleaned)
QDA(normalisation(spotify_data_cleaned),spotify_data_cleaned)
GNB(normalisation(spotify_data_cleaned),spotify_data_cleaned)

LDA :  0.48378717305625224
QDA :  0.5094052310999642
GNB :  0.4753672518810462


1 vs all 6 model
1 vs 1 18 model

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c2ad4461-7f35-42f5-b2d9-f686e8952f67' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>